In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

In [1]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
from keras.models import load_model
 
# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    #print (f"TRIAN Y\n {trainY}, \n\n\nTEST Y\n{testY}")
    
    # print(len(trainX))
    # print(len(trainY))

    # print(len(testX))
    # print(len(testY))

    # reshape dataset to have a single color channel
    # trainX.shape is 60000,28,28
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    # trainX.shape is 60000,28,28,1
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    
    # one hot encode target values
    trainY = to_categorical(trainY)

    # for x in trainY[1]:
    #     print (x)
    testY = to_categorical(testY)
    # print (f"TRAIN X{trainX},\n\n\nTRIAN Y {trainY}, \n\n\nTEST X{testX}, \n\n\nTEST Y{testY}")
    return trainX, trainY, testX, testY

Using TensorFlow backend.


In [2]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm


In [3]:
# define cnn model
def define_model():
    model = Sequential()
    #32 filters, each 3x3
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    
    #batch normalization (turns out to not do much) -from the improvement to learning section
    model.add(BatchNormalization())

    #2x2 max pooling layer
    model.add(MaxPooling2D((2, 2)))

    #more filters -from the increase in model depth section
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))

    #flatten the pools
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
   
    #batch normalization (turns out to not do much) -from the improvement to learning section
    model.add(BatchNormalization())

    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    #print(type(model))
    return model

In [4]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    # print(len(dataX))
    # print(dataX[0][0][0])
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
		# select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=1)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=1)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories

In [5]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(2, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(2, 1, 2)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

In [6]:
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

In [7]:
# run the test harness for evaluating a model
def run_test_harness():
	trainX, trainY, testX, testY = load_dataset()
	trainX, testX = prep_pixels(trainX,testX)
	scores, histories = evaluate_model(trainX,trainY)
	print (scores, histories)
	summarize_diagnostics(histories)
	summarize_performance(scores)

In [ ]:
# from the Save Final Model sections
def run_test_harness_save():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
	# save model
	model.save('final_model.h5')

In [ ]:
# from the Evaluate Final Model section
def run_test_harness_eval():
    # load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# load model
	model = load_model('final_model.h5')
	# evaluate model on test dataset
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))

In [9]:

run_test_harness()
# run_test_harness_save()
# run_test_harness_eval()



Train on 48000 samples, validate on 12000 samples
Epoch 1/10
33216/48000 [===================>..........] - ETA: 23s - loss: 0.1388 - accuracy: 0.9580

KeyboardInterrupt: 

In [8]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0
